# Домашнее задание №3
## Знакомство с языком обработки данных SAS Base
### Транспонирование. SAS Macro

_[Гончаренко Дмитрий Александрович, Жуков Павел Николаевич, Слабоузова Анна Константиновна]_

### 0. Подготовка файлов для выполнения ДЗ.
Запустите код ниже, чтобы создать набор данных SAS в библиотеке **WORK** для выполнения данного задания. Обратите внимание на операторы шага DATA: `infile`, `input`, `cards`. 

In [75]:
****************************************************;
* Create file1 input file;
****************************************************;
data work.narrow_file1;
    infile cards;
    length pet_owner $10 pet $4 population 4;
    input pet_owner $1-10 pet $ population;
cards;
Mr. Black dog 2
Mr. Black bird 1
Mrs. Green fish 5
Mr. White cat 3
;
run;

### 1. Транспонирование наборов данных в SAS 

Транспонирование оказывается очень полезным, когда мы хотим преобразовать данные к формату пригодному для анализа. 
Иногда из таблицы, где на одного клиента приходится несколько строк с разными характеристиками, нужно сделать таблицу, где на клиента одна строка, а его характеристики - это разные столбцы. 
- Например, из таблицы:

| Client | Account |  Balance |
|:---:|:---:|:---:|
| Smith | checking | 1,000.00 |
| Smith | savings | 4,000.00 |
| Smith | mortgage | 150,000.00 |
| Smith | credit_card | 500.00|
|Jones|checking | 973.78 |
|Jones| savings | 2,613.44 |
|Jones| mortgage | . |
|Jones|credit_card |  140.48 |

 - Сделать таблицу: 

|Client| checking| savings| mortgage| credit_card|
|:---:|:---:|:---:|:---:|:---:|
|Smith|	 1,000.00| 4,000.00 | 150,000.00| 500.00|
|Jones | 973.78 | 2,613.44 | . |140.48|


Выполнив примеры из этой части, вы за 30 мин познакомитесь с основами транспонирования в SAS.

Документацию по процедуре ** _proc transpose_ ** можно и нужно посмотреть [здесь](http://documentation.sas.com/?docsetId=proc&docsetVersion=9.4&docsetTarget=n1xno5xgs39b70n0zydov0owajj8.htm&locale=en)

### 1.1   Самый простой случай

Если в proc transpose указать только входной и выходной набор данных, то поумолчанию транспонируются только числовые переменные.  

In [7]:
title 'набор до транспонирования';
proc print data=narrow_file1; run;

proc transpose data=narrow_file1 out=transposed_narrow_file1; run;

title 'набор после транспонирования';
proc print data=transposed_narrow_file1; run;
title; 

**Обратите внимание:**
- строковые переменные `pet_owner` и `pet` просто проигнорированы, как будто во входной таблице была только одна переменная `population`!
- не проводя сравнение результата и входных данных, трудно сказать, что конкретно выведено в переменной `COL1`!
- а что в переменной `_NAME_`?


### 1.2 Опция `prefix=` и `name=` 
- Чтобы результат был понятнее, к именан выходных переменных бывает полезно добавить _префикс_. 
- `name=` позволяет явно задать имя для переменной, содержащей название транспонированной колонки.  

In [8]:
proc transpose data=narrow_file1 
               out=transposed_narrow_file1
               prefix=pet_count
               name=SOURCE
               ; 
run;

title 'набор после транспонирования (опция prefix= и name=)';
proc print data=transposed_narrow_file1; run;
title; 

Obs,SOURCE,pet_count1,pet_count2,pet_count3,pet_count4
1,population,2,1,5,3


### 1.3 Оператор `id ...;`
В операторе можно указать переменную **_входного_** набора, чьи **_значения_** станут названиями переменных в **_выходном_** наборе.
 - понятно, что **_значения должны быть различными _**- не может же быть двух переменных с одинаковыми названиями. 
 - опцию `prefix=` можно не использовать, если значения могут стать именами (т.е. начинаться с буквы, состоять из букв и цифр и пр.)

In [9]:
title 'набор до транспонирования';
proc print data=narrow_file1; run;

proc transpose data=narrow_file1 
               out=transposed_narrow_file1
               name=SOURCE
               ; 
    id pet;
run;

title 'набор после транспонирования (оператор id ...;)';
proc print data=transposed_narrow_file1; run;
title; 

### 1.4  Транспонирование нескольких переменных
Для этого можно явно указать список переменных для транспонирования в операторе `var ...;`  

In [10]:
proc transpose data=narrow_file1 
               out=transposed_narrow_file1
               name=SOURCE
               ; 
    var pet population;
run;

title 'набор после транспонирования (оператор var ...;)';
proc print data=transposed_narrow_file1; run;
title; 

Obs,SOURCE,COL1,COL2,COL3,COL4
1,pet,dog,bird,fish,cat
2,population,2,1,5,3


### 1.5  Транспонирование внутри группы - оператор `by...;`
Если мы хотим получить по одному наблюдению на владельца (pet_owner), то используем данный оператор. 

**_Замечание:_** набор должен быть отсортирован по переменным, указанным в `by ...;` или нужно явно сказать, что набор не был сортирован по данной переменной (см. ниже)!    

**_Замечание:_** используйте оператор `id ...;` вместе с `by ...;`, чтобы создавались переменные для всех различных значений во всех группах! (Попробуйте закоментировать оператор `id ...;` - посмотрите на результат - не совсем то, что хотелось?)   


In [11]:
proc transpose data=narrow_file1 
               out=transposed_narrow_file1
               name=SOURCE
               ; 
    id pet;
    var population;
    by notsorted pet_owner;
run;

title 'набор после транспонирования (оператор by ...;)';
proc print data=transposed_narrow_file1; run;
title; 

Obs,pet_owner,SOURCE,dog,bird,fish,cat
1,Mr. Black,population,2,1,.,.
2,Mrs. Green,population,.,.,5,.
3,Mr. White,population,.,.,.,3


### 1.6 В обратную сторону
В общем, то же самое:
 - `var ...;` перечисление переменных для транспонирования (иначе будут использованы только числовые); 
 - `by ...;` говорит, что процесс происходит внутри групп (не сортированных);
 - `name=` задает имя переменной - результата транспонирования;
 - `prefix=` - префикс;
 
 _**Обратите внимание:**_ кол-во наблюдений в группах по `pet_owner` всегда одинаковое и равно кол-ву переменных в `var ...;`

In [12]:
proc transpose data=transposed_narrow_file1 
               out=back_narrow_file1
               name=pet
               prefix=population
               ; 
    var dog bird fish cat;
    by notsorted pet_owner;
run;

title 'набор после транспонирования';
proc print data=back_narrow_file1; run;
title; 

Obs,pet_owner,pet,population1
1,Mr. Black,dog,2
2,Mr. Black,bird,1
3,Mr. Black,fish,.
4,Mr. Black,cat,.
5,Mrs. Green,dog,.
6,Mrs. Green,bird,.
7,Mrs. Green,fish,5
8,Mrs. Green,cat,.
9,Mr. White,dog,.
10,Mr. White,bird,.


#### 1.6.1 **ЗАДАНИЕ:** 
Используя опции наборов данных, 1) переименуйте population1 в population, 2) удалите все строки с population1 = .;

In [13]:
proc transpose data=transposed_narrow_file1 
               out=back_narrow_file1
               name=pet
               prefix=population
               ; 
    var dog bird fish cat;
    by notsorted pet_owner;
run;

data work.changed;
set back_narrow_file1;
Rename population1 = population;
if cmiss(of _all_) then delete;
run;

title 'набор после транспонирования';
proc print data=work.changed; run;
title; 

Obs,pet_owner,pet,population
1,Mr. Black,dog,2
2,Mr. Black,bird,1
3,Mrs. Green,fish,5
4,Mr. White,cat,3


### 2. SAS Macro
Вам нужно дописать макрос, определенный ниже, который производит транспонирование набора данных (передается в параметре ds=) по переменным из списка (парамерт vars=), переменные в списке разделены пробелами, и печатает результат. Транспонирование самое простое: указываем входной и выходной набобры, ну и переменные в операторе var=...; 
Макрос должен:
 - корректно обрабатывать ситуации, когда в макрос переданы пустые параметры;
 - проверять существует ли файл, указанные в параметре ds=, если нет, то печатать в лог соответствующее сообщение;
 - проверять присутствуют ли в наборе данных ds= переменные из списка vars=, транспонировать только по тем что есть;
 - корректно обрабативать ситуация, когда в списке vars= нет ни одной переменной из набора ds=;

** После того, как написали макрос, выполните 5 вызовов макроса ниже.**
 
**_Замечание:_** вам понадобиться %sysfunc(), чтобы вызвать необходимые для решения задачи функции SAS.

In [85]:
/* определение макроса */
OPTIONS NOSYMBOLGEN;
OPTIONS NOMPRINT;
OPTIONS NOMLOGIC;
OPTIONS MCOMPILENOTE=none;
%macro transpose_dataset_vars(ds=, vars=);
    %local rc dcid real_vars var i;
    /*  Проверка на пустоту параметров  */
    %if &ds ne and &vars ne %then %do;

        /*  Существует ли dataset?  */
        %if %sysfunc(exist(&ds)) %then %do;
        
            /*  Открываетм файл ds  */
            %let dsid = %sysfunc(open(&ds));
            
            /*  Существующие переменные vars в ds  */
            %let real_vars=%str( );

            /*  Итеративно пробегаем по всем параметрам vars  */
            %let i = 1;
            %let var = %scan(&vars, &i, %str( ));
            %do %while(&var ne);
            
                /*  Существует ли var в ds  */
                %if %sysfunc(varnum(&dsid, &var)) > 0 %then %do;
                    /*  Добавим его к real_vars  */
                    %let real_vars = %sysfunc(catx(%str( ), &real_vars, &var));
                %end;

                %let i = %eval(&i + 1);
                %let var=%scan(&vars, &i, %str( ));
            %end;
            
            /*  Закрываем ds  */
            %let rc = %sysfunc(close(&dsid));

            /* Вывод транспонирования  */
            %if &real_vars ne %then %do;
                   proc transpose data=&ds 
                               out=out_table
                               ; 
                    var &real_vars;
                run;
                proc print data=out_table; run;
            %end;
            %else %put "No such variables in &ds";
        %end;
        %else %put "file &ds not existing";
    %end;
    %else %put "empty parametrs";
run;

%mend transpose_dataset_vars;

In [86]:
/* вызов макроса 1 */
%transpose_dataset_vars(ds=narrow_file1, vars=pet population );

Obs,_NAME_,COL1,COL2,COL3,COL4
1,pet,dog,bird,fish,cat
2,population,2,1,5,3


In [87]:
/* вызов макроса 2 */
%transpose_dataset_vars(ds=narrow_1file1, vars=pet population);

In [88]:
/* вызов макроса 3 */
%transpose_dataset_vars(ds=narrow_file1, vars=pet population age);

Obs,_NAME_,COL1,COL2,COL3,COL4
1,pet,dog,bird,fish,cat
2,population,2,1,5,3


In [89]:
/* вызов макроса 4 */
%transpose_dataset_vars(ds=narrow_file1, vars=);

In [90]:
/* вызов макроса 5 */
%transpose_dataset_vars(ds=narrow_file1, vars=age city);